In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import pprint as pp
import math
import collections
#from ml_metrics import rmsle

df_train = pd.read_csv('../data/macroCleanEngineeredTrain.csv')
df_test = pd.read_csv('../data/macroCleanEngineeredTest.csv')

/home/vito/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/vito/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def rmsle(preds, dtrain):
	labels = dtrain.get_label()
	assert len(preds) == len(labels)
	labels = labels.tolist()
	preds = preds.tolist()
	terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0, preds[i]) + 1)) ** 2.0 for i, pred in enumerate(labels)]
	return 'rmsle', (sum(terms_to_sum) * (1.0 / len(preds))) ** 0.5

In [14]:
num_features = 200
importances = collections.OrderedDict()
with open('../preprocessing/rfFeatureImportance.txt', 'r') as f:
    for line in f:
        parts = line.split(' ')
        importances[parts[1]] = parts[2].lstrip('(').rstrip(')\n')
        
selected_features = set(importances.keys()[0:num_features])
print str(selected_features)

set(['office_sqm_1500', 'cafe_count_3000_na_price', 'trc_count_1000', 'cafe_sum_1000_max_price_avg', 'cafe_count_500_price_1000', 'railroad_station_avto_min', 'office_sqm_5000', 'trc_sqm_2000', 'cafe_count_5000_price_2500', 'trc_sqm_3000', 'office_sqm_500', 'office_sqm_1000', 'cafe_sum_2000_max_price_avg', 'build_count_brick', 'cafe_count_1000', 'big_church_count_2000', 'max_floor', 'church_count_1000', 'office_count_3000', 'cafe_avg_price_3000', 'cafe_count_2000_na_price', 'trc_count_5000', 'market_count_3000', 'catering_km', 'zd_vokzaly_avto_km', 'sport_count_5000', 'trc_sqm_1500', 'water_km', 'green_part_1000', 'ID_railroad_station_walk', 'cafe_sum_500_max_price_avg', 'museum_km', 'material', 'park_km', 'railroad_station_walk_min', 'cafe_avg_price_5000', 'hospice_morgue_km', 'metro_km_avto', 'preschool_quota', 'radiation_km', 'public_transport_station_min_walk', 'cafe_count_1000_price_1000', 'build_count_foam', 'school_quota', 'public_healthcare_km', 'ice_rink_km', 'prom_part_2000',

In [15]:
# We take all float/int columns except for ID, timestamp, and the target value
train_columns = list(
	set(df_train.select_dtypes(include=['float64', 'int64']).columns) - set(['id', 'timestamp', 'price_doc']) \
    & selected_features)

y_train = df_train['price_doc'].values
x_train = df_train[train_columns].values
x_test = df_test[train_columns].values

In [16]:
# Train/Valid split
split = 25000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 800, watchlist, feval=rmsle, early_stopping_rounds=100)

p_test = clf.predict(xgb.DMatrix(x_test))

sub = pd.DataFrame()
sub['id'] = df_test['id'].values
sub['price_doc'] = p_test
sub.to_csv('xgb.csv', index=False)

[0]	train-rmse:8.74623e+06	valid-rmse:9.58156e+06	train-rmsle:3.85687	valid-rmsle:3.92767
Multiple eval metrics have been passed: 'valid-rmsle' will be used for early stopping.

Will train until valid-rmsle hasn't improved in 100 rounds.
[1]	train-rmse:8.58961e+06	valid-rmse:9.43179e+06	train-rmsle:3.18086	valid-rmsle:3.25184
[2]	train-rmse:8.43625e+06	valid-rmse:9.28468e+06	train-rmsle:2.79036	valid-rmsle:2.8612
[3]	train-rmse:8.2864e+06	valid-rmse:9.14029e+06	train-rmsle:2.51717	valid-rmsle:2.58711
[4]	train-rmse:8.13972e+06	valid-rmse:9.00125e+06	train-rmsle:2.30816	valid-rmsle:2.37821
[5]	train-rmse:7.99607e+06	valid-rmse:8.86396e+06	train-rmsle:2.13914	valid-rmsle:2.20935
[6]	train-rmse:7.85575e+06	valid-rmse:8.72955e+06	train-rmsle:1.99906	valid-rmsle:2.06861
[7]	train-rmse:7.71838e+06	valid-rmse:8.59986e+06	train-rmsle:1.87923	valid-rmsle:1.94886
[8]	train-rmse:7.58382e+06	valid-rmse:8.47047e+06	train-rmsle:1.77522	valid-rmsle:1.8436
[9]	train-rmse:7.45224e+06	valid-rmse:8.34592